In [1]:
# ╭──────────────────────────────────────────────╮
# │ 1. 加载 API Keys                             │
# ╰──────────────────────────────────────────────╯

%load_ext dotenv
%dotenv

import os
from dotenv import load_dotenv
load_dotenv()

QWEN_API_KEY = os.getenv("QWEN_API_KEY")
if not QWEN_API_KEY:
    raise ValueError("❌ 请配置 QWEN_API_KEY")

print("✅ Qwen API Key 已加载")

✅ Qwen API Key 已加载


In [3]:
# ╭──────────────────────────────────────────────╮
# │ 2. 安装必要库                                │
# │ （仅用于 PDF/文本处理）                      │
# ╰──────────────────────────────────────────────╯

# !pip install PyPDF2 sentence-transformers numpy scikit-learn

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
import re

In [4]:
# ╭──────────────────────────────────────────────╮
# │ 3. 创建示例知识库                            │
# ╰──────────────────────────────────────────────╯

knowledge_base = [
    "阿里巴巴集团成立于1999年，总部位于中国杭州。",
    "通义千问（Qwen）是阿里巴巴推出的超大规模语言模型，支持多轮对话和多种语言。",
    "阿里云是全球知名的云计算及人工智能科技公司，提供弹性计算、数据库、安全等服务。",
    "2024年，阿里巴巴发布了Qwen2和Qwen-Max模型，性能大幅提升。",
    "阿里巴巴的使命是让天下没有难做的生意。"
]

# 分块（这里每句已是一个 chunk）
chunks = knowledge_core = knowledge_base
print("📚 知识库 chunks:")
for i, c in enumerate(chunks):
    print(f"  {i}: {c}")

📚 知识库 chunks:
  0: 阿里巴巴集团成立于1999年，总部位于中国杭州。
  1: 通义千问（Qwen）是阿里巴巴推出的超大规模语言模型，支持多轮对话和多种语言。
  2: 阿里云是全球知名的云计算及人工智能科技公司，提供弹性计算、数据库、安全等服务。
  3: 2024年，阿里巴巴发布了Qwen2和Qwen-Max模型，性能大幅提升。
  4: 阿里巴巴的使命是让天下没有难做的生意。


In [ ]:
# ╭──────────────────────────────────────────────╮
# │ 4. 调用 Qwen Embedding 模型                   │
# │ 注意：DashScope 兼容 OpenAI 的 embeddings 接口  |
# │ 在guides下有Embeddings简单介绍。                │
# ╰──────────────────────────────────────────────╯

embedding_client = OpenAI(
    api_key=QWEN_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

def get_embedding(text: str) -> list[float]:
    response = embedding_client.embeddings.create(
        model="text-embedding-v2",  # 阿里云 embedding 模型名
        input=text
    )
    return response.data[0].embedding

# 为所有 chunks 生成 embedding（实际应用中应缓存！）
chunk_embeddings = [get_embedding(chunk) for chunk in chunks]
print(f"✅ 已生成 {len(chunk_embeddings)} 个 chunk 的 embedding")

✅ 已生成 5 个 chunk 的 embedding


In [6]:
# ╭──────────────────────────────────────────────╮
# │ 5. 相似性检索                                │
# ╰──────────────────────────────────────────────╯

def retrieve(query: str, top_k: int = 2) -> list[str]:
    query_emb = np.array(get_embedding(query)).reshape(1, -1)
    chunk_embs = np.array(chunk_embeddings)
    
    similarities = cosine_similarity(query_emb, chunk_embs)[0]
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    
    return [chunks[i] for i in top_indices]

# 测试检索
query = "Qwen 是哪家公司发布的？"
retrieved = retrieve(query)
print(f"🔍 查询: {query}")
print("📄 检索到的相关片段:")
for r in retrieved:
    print(f"  • {r}")

🔍 查询: Qwen 是哪家公司发布的？
📄 检索到的相关片段:
  • 2024年，阿里巴巴发布了Qwen2和Qwen-Max模型，性能大幅提升。
  • 通义千问（Qwen）是阿里巴巴推出的超大规模语言模型，支持多轮对话和多种语言。


In [7]:
# ╭──────────────────────────────────────────────╮
# │ 6. 生成最终回答                              │
# ╰──────────────────────────────────────────────╯

llm_client = OpenAI(
    api_key=QWEN_API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

def rag_answer(question: str) -> str:
    # 1. 检索
    docs = retrieve(question, top_k=2)
    context = "\n".join(docs)
    
    # 2. 构造 prompt
    prompt = f"""
你是一个准确、可靠的信息助手。
请**严格基于以下提供的资料**回答问题，不要编造。

资料：
{context}

问题：{question}
    """.strip()
    
    # 3. 调用 LLM
    response = llm_client.chat.completions.create(
        model="qwen-max",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1  # 降低随机性，提高事实性
    )
    return response.choices[0].message.content

# 测试
question = "Qwen 是哪家公司发布的？"
answer = rag_answer(question)
print(f"❓ 问题: {question}")
print(f"🤖 回答: {answer}")

❓ 问题: Qwen 是哪家公司发布的？
🤖 回答: Qwen 是由阿里巴巴发布的。


In [8]:
# ╭──────────────────────────────────────────────╮
# │ 7. 对比实验                                  │
# ╰──────────────────────────────────────────────╯

def direct_answer(question: str) -> str:
    response = llm_client.chat.completions.create(
        model="qwen-max",
        messages=[{"role": "user", "content": question}],
        temperature=0.7
    )
    return response.choices[0].message.content

question = "阿里巴巴成立于哪一年？"

print("🧪 对比实验：")
print(f"问题: {question}\n")

print("❌ 无 RAG（可能幻觉）:")
print(direct_answer(question))

print("\n✅ 有 RAG（基于知识库）:")
print(rag_answer(question))

🧪 对比实验：
问题: 阿里巴巴成立于哪一年？

❌ 无 RAG（可能幻觉）:
阿里巴巴集团成立于1999年。

✅ 有 RAG（基于知识库）:
阿里巴巴集团成立于1999年。
